In [1]:
#from werkzeug.security import generate_password_hash
#print(generate_password_hash("test"))

In [2]:
import sqlite3
import pandas as pd

conn = sqlite3.connect("database.db")
cur = conn.cursor()

In [3]:
#create munros table
create_munro_table = """CREATE TABLE munros (
                    munro_id TEXT PRIMARY KEY,
                    name TEXT UNIQUE,
                    region TEXT,
                    height TEXT,
                    whl_url TEXT,
                    latitude FLOAT,
                    longitude FLOAT
                );
                """
cur.execute("DROP TABLE IF EXISTS munros;")
cur.execute(create_munro_table)

#add data from csv
munro_df = pd.read_csv("data/munros.csv")
munro_df.to_sql("munros", conn, if_exists="append", index=False)
conn.commit()

In [4]:
#create users table
create_user_table = """CREATE TABLE users (
                    id INTEGER PRIMARY KEY AUTOINCREMENT,
                    user_id TEXT,
                    user_name TEXT UNIQUE NOT NULL,
                    password TEXT NOT NULL
                );
                """

cur.execute("DROP TABLE IF EXISTS users;")
cur.execute(create_user_table)


user_df = pd.read_csv("data/users.csv")
user_df.to_sql("users", conn, if_exists="append", index=False)
conn.commit()

In [5]:
#create junction table
#add junction bag table
create_junction_table = """CREATE TABLE bags (
                        user_id INTEGER NOT NULL,
                        munro_id INTEGER NOT NULL,
                        date TEXT NOT NULL,
                        distance TEXT,
                        friends TEXT,
                        notes TEXT,
                        private TEXT,
                        FOREIGN KEY (user_id) REFERENCES users(user_id),
                        FOREIGN KEY (munro_id) REFERENCES munros(munro_id)
                        );
                        """

cur.execute("DROP TABLE IF EXISTS bags;")
cur.execute(create_junction_table)

bag_df = pd.read_csv("data/bags.csv")
bag_df.to_sql("bags", conn, if_exists="append", index=False)
conn.commit()

In [6]:
#create teams table
create_team_table = """CREATE TABLE teams (
                    id INTEGER PRIMARY KEY AUTOINCREMENT,
                    team_id TEXT,
                    team_name TEXT UNIQUE NOT NULL
                    );
                    """

cur.execute("DROP TABLE IF EXISTS teams;")
cur.execute(create_team_table)


team_df = pd.read_csv("data/teams.csv")
team_df.to_sql("teams", conn, if_exists="append", index=False)
conn.commit()

In [7]:
#create members table
create_member_table = """CREATE TABLE members (
                    team_id TEXT NOT NULL,
                    user_id TEXT NOT NULL,
                    UNIQUE (team_id, user_id)
                );
                """

cur.execute("DROP TABLE IF EXISTS members;")
cur.execute(create_member_table)


member_df = pd.read_csv("data/members.csv")
member_df.to_sql("members", conn, if_exists="append", index=False)
conn.commit()

In [12]:
conn.close()

In [9]:
def get_team_bags(team_name):

    #get all bags for team members
    team_bags_query = """SELECT user_id, munro_id
                        FROM bags 
                        WHERE user_id IN (
                            SELECT user_id 
                            FROM members 
                            WHERE team_id = (
                                SELECT team_id 
                                FROM teams 
                                WHERE team_name = ?
                            ));
                        """

    #group users by munro
    team_df = pd.read_sql(team_bags_query, conn, params=(team_name,))
    team_df = team_df.groupby("munro_id")["user_id"].agg(list).reset_index()

    #join on munro data
    munro_df = pd.read_sql("SELECT * FROM munros", conn)
    team_data_df = munro_df.merge(team_df, how="left", on="munro_id")
    team_data_df["user_id"] = team_data_df["user_id"].apply(lambda x: x if isinstance(x, list) else [])

    #add user name
    user_df = pd.read_sql("SELECT * FROM users", conn)
    user_name_dict = dict(zip(user_df["user_id"], user_df["user_name"]))
    team_data_df["user_name"] = team_data_df["user_id"].apply(lambda lst: [user_name_dict.get(x, None) for x in lst])

    #count of bags per munro
    team_data_df["count"] = team_data_df["user_id"].apply(lambda x: len(x))

    #drop user_name
    team_data_df = team_data_df.drop(columns="user_id")

    return team_data_df

In [10]:
team_df = get_team_bags("Scott's hos")

In [11]:
team_df

,munro_id,name,region,height,whl_url,latitude,longitude,user_name,count
0,m000,Ben Nevis,Fort William,1345m,https://www.walkhighlands.co.uk/munros/ben-nevis,56.796891,-5.003675,[],0
1,m001,Ben Macdui,Cairngorms,1309m,https://www.walkhighlands.co.uk/munros/ben-macdui,57.070368,-3.669099,"[matthew, amy]",2
2,m002,Braeriach,Cairngorms,1296m,https://www.walkhighlands.co.uk/munros/braeriach,57.078298,-3.728373,[],0
3,m003,Cairn Toul,Cairngorms,1291m,https://www.walkhighlands.co.uk/munros/cairn-toul,57.054406,-3.710757,[],0
4,m004,Sgòr an Lochain Uaine,Cairngorms,1258m,https://www.walkhighlands.co.uk/munros/sgor-an...,57.058376,-3.725897,[],0
...,...,...,...,...,...,...,...,...,...
277,m277,Meall na Teanga,Fort William,918m,https://www.walkhighlands.co.uk/munros/meall-n...,56.989030,-4.930940,[],0
278,m278,Beinn a' Chlèibh,Argyll,916m,https://www.walkhighlands.co.uk/munros/beinn-a...,56.390230,-4.835630,[],0
279,m279,Ben Vane,Loch Lomond,915m,https://www.walkhighlands.co.uk/munros/ben-vane,56.249786,-4.781655,[amy],1
280,m280,Càrn Aosda,Cairngorms,917m,https://www.walkhighlands.co.uk/munros/carn-aosda,56.895696,-3.423274,"[matthew, amy]",2
